In [ ]:
import os
import sys
import urllib.request
import pandas as pd
import json
from bs4 import BeautifulSoup
import re
import requests
from tabulate import tabulate

def BAR():
    print('--------------------------------------------------------------------')

def print_DF(df):
    print(tabulate(df, headers='keys', tablefmt='psql'))

#네이버 api를 통해 검색 후 상위 5개 검색어 반환
def searching(title):
    client_id = "JnkicuvSa2phrxPmgIXM"
    client_secret = "7nhnLrKQWp"
    
    encText = urllib.parse.quote(title)
    
    url = "https://openapi.naver.com/v1/search/book?query=" + encText +"&display=5"

    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id",client_id)
    request.add_header("X-Naver-Client-Secret",client_secret)
    response = urllib.request.urlopen(request)
    
    rescode = response.getcode()
    if(rescode == 200):
        response_body = response.read()
        result = response_body.decode('utf-8')
        result = json.loads(result)

        print("Success!")
        return result
    else:
        print("Error Code:" + rescode)
        return -1

#상위 5개의 검색 결과에서 사용자가 원하는 책을 고릅니다.
def choose_one_return_url(book):
    book_df = pd.DataFrame(book['items'])

    for i in range(len(book_df)):
        book_df.iloc[i]['title'] = BeautifulSoup(book_df.iloc[i]['title'], "html5lib").get_text()
    
    book_to_choice = book_df[['title','author']]

    print_DF(book_to_choice)   

    while 1:
        choice = int(input('Please enter the number of the book you want.: '))

        if choice < 0 or choice >= 5:
            print("Please enter 0 between 5.")
        else:
            break        
    print("Success!")

    return book_df.iloc[choice]["title"], book_df.iloc[choice]["link"]

#사용자가 선택한 책의 정보 전체를 받아옴
def make_description(URL):   
    url = requests.get(URL)

    #BeautifulSoup: requests로 받아온 html 문서를 보기 좋게 꾸며주는 틀
    soup = BeautifulSoup(url.text, 'html.parser')

    whole_review = soup.findAll("div", attrs={"id":"bookIntroContent"})
    whole_review

    for line in whole_review:
        book_des = line.get_text()
    print("Success!")
    return book_des

#한글 -> 영어
def translation(target):
    client_id = "9IHX5zHlL1QnIXwNAxGk"
    client_secret = "WbabULoTuj"

    url = "https://openapi.naver.com/v1/papago/n2mt"

    #번역할 언어와 내용에 대해
    text = target
    encText = urllib.parse.quote(text)
    srcLang = 'ko'
    tarLang = 'en'
    data = "source={}&target={}&text=".format(srcLang,tarLang) + encText

    #웹 요청
    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id",client_id)
    request.add_header("X-Naver-Client-Secret",client_secret)

    #결과 받아오는 부분
    response = urllib.request.urlopen(request, data=data.encode("utf-8"))

    #응답
    rescode = response.getcode()
    if(rescode==200): #성공
        response_body = response.read()
        data = response_body.decode('utf-8')
        data = json.loads(data) #딕셔너리화
        trans_text = data['message']['result']['translatedText']
        trans_text = re.sub('[^A-Za-z0-9]+', ' ', trans_text)
        return trans_text
    else: #실패
        print("Error Code:" + rescode)
        return -1

In [ ]:
user_data = input("Please enter the title of the book : ")
result = searching(user_data)
book_title, url = choose_one_return_url(result)
book_des = make_description(url)

book_title = translation(book_title)
book_des = translation(book_des)

BAR()
print('Book Title: ', book_title)
print('Book Description: ', book_des)


Please enter the title of the book : 악의
Success!
+----+----------------------------------------------------------------------------------------+----------------------+
|    | title                                                                                  | author               |
|----+----------------------------------------------------------------------------------------+----------------------|
|  0 | 악의 (가가 형사 시리즈)                                                                | 히가시노 게이고      |
|  1 | 악의 평범성 (이산하 시집)                                                              | 이산하               |
|  2 | 악의 꽃 (앙리 마티스 에디션)                                                           | 샤를 피에르 보들레르 |
|  3 | 악의 마음을 읽는 자들 (국내 최초 프로파일러의 연쇄살인 추적기)                         | 권일용|고나무        |
|  4 | 초판본 악의 꽃 - 1857년 오리지널 초판본 표지디자인 (1857년 오리지널 초판본 표지디자인) | 샤를 보들레르        |
+----+----------------------------------------------------------------------------------------+----------------------+

In [ ]:
#주피터 노트북이 아닌 Colab에서 돌릴 경우 구글 드라이브에서 해당 파일 불러오는 부분
from google.colab import drive
drive.mount('/content/drive')

file = "/content/drive/MyDrive/Colab Notebooks/movies_metadata.csv"

data = pd.read_csv(file)
data = data[['title','overview','imdb_id']]
#Colab 아니면 아래 코드 이용
#data = pd.read_csv("movies_metadata.csv")

data = data.head(30000)

#영화 데이터 프레임에 책 description 추가
book_data = {'title': book_title, 'overview' : book_des}
data = data.append(book_data,ignore_index=True)


#Null 값 확인
data['overview'].isnull().sum()

# overview에서 NaN 값을 가진 경우를 제거(tf-idf를 할 때 NaN값이 들어있으면 에러가 나기 때문)
#해당 행을 그대로 두고 overivew의 NaN값만 제거
data['overview'] = data['overview'].fillna('')
#Null 값 확인
data['overview'].isnull().sum()

data['overview'].tail()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


29996    A beautiful lonely girl named Melissa tries to...
29997    A journalist starts an investigation into the ...
29998    When an out of work actress unexpectedly lands...
29999    Bumbling navy officer Lieutenant Humphrey Fair...
30000    A brain game that is chased and chased over th...
Name: overview, dtype: object

In [ ]:
data.tail(1)

,title,overview,imdb_id
30000,Malicious GaGaDetective Series,A brain game that is chased and chased over th...,NaN


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(stop_words='english') # 불용어 제거

# overview에 대해서 TF-IDF 수행
tfidf_matrix = tfidf.fit_transform(data['overview'])
# (데이터 개수, 단어 개수)
print(tfidf_matrix.shape)



(30001, 58571)


In [ ]:
from sklearn.metrics.pairwise import linear_kernel

#코사인 유사도 계산
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

#제목에 관한 인덱스 테이블 구축
#책 제목 입력하면 인덱스를 리턴하기 위함
indices = pd.Series(data.index, index=data['title']).drop_duplicates()


In [ ]:
#입력한 책에 대해서 코사인 유사도를 이용하여 overview가 가장 유사한 10개의 영화 찾아냄
def recommendation(title, cosine_sim=cosine_sim):
    # 입력한 책의 인덱스 가져오기
    idx = indices[title]

    # 모든 영화에 대해서 해당 책과의 유사도를 구함
    sim_scores = list(enumerate(cosine_sim[idx]))

    # 유사도에 따라 정렬
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # 가장 유사한 10개의 영화를 받아옴
    sim_scores = sim_scores[1:11]

    # 가장 유사한 10개의 영화의 인덱스를 받아옴
    movie_indices = [i[0] for i in sim_scores]

    #기존에 읽어들인 데이터에서 해당 인덱스의 값들을 가져온다
    #그리고 스코어 열을 추가하여 코사인 유사도도 확인할 수 있게 한다.
    result_df = data.iloc[movie_indices].copy()
    result_df['score'] = [i[1] for i in sim_scores]
    
    # 가장 유사한 10개의 영화의 제목과 스코어 리턴
    result_df = result_df[["title","score",'imdb_id']]
    return result_df

movie_res = pd.DataFrame(recommendation(book_title))

In [ ]:
movie_res

,title,score,imdb_id
6057,Stone Reader,0.144195,tt0324080
23779,The Detective,0.134250,tt1107807
23217,g@me.,0.130656,tt0431077
29594,The Impossible Hour,0.128400,tt0378936
24456,The Bat,0.127942,tt0016629
14408,A Night to Remember,0.124207,tt0036203
24466,Young Detective Dee: Rise of the Sea Dragon,0.116997,tt2992146
13448,The Unsuspected,0.115388,tt0039941
19604,"Farewell, My Queen",0.109917,tt1753813
26124,The Wizard of Gore,0.109892,tt0765487


In [ ]:
from imdb import IMDb

# create an instance of the IMDb class
ia = IMDb()
def get_img(id):
    mv = ia.get_movie(re.sub(r'[^0-9]', '',id))
    URL = ia.get_imdbURL(mv) #URL for first result 
    # getting cover url of the series
    cover = mv.data['cover url']

    data = {'image': cover, 'URL' : URL}
    return data

In [ ]:
df_movie = pd.DataFrame()

for i in range(len(movie_res)) :
    data = get_img(movie_res.iloc[i]['imdb_id'])
    df_temp = pd.DataFrame(data, index = [movie_res.index[i]])
    df_movie = pd.concat([df_movie, df_temp])
    


In [ ]:
df_movie = pd.concat([movie_res, df_movie ], axis=1)

df_movie

,title,score,imdb_id,image,URL
6057,Stone Reader,0.144195,tt0324080,https://m.media-amazon.com/images/M/MV5BMTgzOD...,https://www.imdb.com/title/tt0324080/
23779,The Detective,0.134250,tt1107807,https://m.media-amazon.com/images/M/MV5BMjM5NT...,https://www.imdb.com/title/tt1107807/
23217,g@me.,0.130656,tt0431077,https://m.media-amazon.com/images/M/MV5BMGNkYj...,https://www.imdb.com/title/tt0431077/
29594,The Impossible Hour,0.128400,tt0378936,https://m.media-amazon.com/images/M/MV5BYTZmMW...,https://www.imdb.com/title/tt0378936/
24456,The Bat,0.127942,tt0016629,https://m.media-amazon.com/images/M/MV5BMTgxMz...,https://www.imdb.com/title/tt0016629/
14408,A Night to Remember,0.124207,tt0036203,https://m.media-amazon.com/images/M/MV5BY2IzM2...,https://www.imdb.com/title/tt0036203/
24466,Young Detective Dee: Rise of the Sea Dragon,0.116997,tt2992146,https://m.media-amazon.com/images/M/MV5BMmI5ND...,https://www.imdb.com/title/tt2992146/
13448,The Unsuspected,0.115388,tt0039941,https://m.media-amazon.com/images/M/MV5BMDY0MT...,https://www.imdb.com/title/tt0039941/
19604,"Farewell, My Queen",0.109917,tt1753813,https://m.media-amazon.com/images/M/MV5BMjI0OT...,https://www.imdb.com/title/tt1753813/
26124,The Wizard of Gore,0.109892,tt0765487,https://m.media-amazon.com/images/M/MV5BMTg2MD...,https://www.imdb.com/title/tt0765487/


In [ ]:
print('Movie recommendation result similar to '+user_data)
from IPython.core.display import HTML
def path_to_image_html(path):
    return '<img src="'+ path + '" width="100" >'

df_movie.to_html(escape=False, formatters=dict(image=path_to_image_html))

HTML(df_movie.to_html(escape=False,formatters=dict(image=path_to_image_html)))

Movie recommendation result similar to 악의


,title,score,imdb_id,image,URL
6057,Stone Reader,0.144195,tt0324080,,https://www.imdb.com/title/tt0324080/
23779,The Detective,0.134250,tt1107807,,https://www.imdb.com/title/tt1107807/
23217,g@me.,0.130656,tt0431077,,https://www.imdb.com/title/tt0431077/
29594,The Impossible Hour,0.128400,tt0378936,,https://www.imdb.com/title/tt0378936/
24456,The Bat,0.127942,tt0016629,,https://www.imdb.com/title/tt0016629/
14408,A Night to Remember,0.124207,tt0036203,,https://www.imdb.com/title/tt0036203/
24466,Young Detective Dee: Rise of the Sea Dragon,0.116997,tt2992146,,https://www.imdb.com/title/tt2992146/
13448,The Unsuspected,0.115388,tt0039941,,https://www.imdb.com/title/tt0039941/
19604,"Farewell, My Queen",0.109917,tt1753813,,https://www.imdb.com/title/tt1753813/
26124,The Wizard of Gore,0.109892,tt0765487,,https://www.imdb.com/title/tt0765487/
